In [ ]:
import pandas as pd
area = pd.read_csv('newtables/area_match_info.csv')
airinfo = pd.read_csv('newtables/air_store_info_new.csv')
hpginfo = pd.read_csv('newtables/hpg_store_info_new.csv')
re = pd.read_csv('restaurant/store_id_relation.csv')
genre = pd.read_csv('newtables/genre_match_info.csv')
air_res_vis_store = pd.read_csv('newtables/air_res_vis_store.csv')
hpg_res_store = pd.read_csv('newtables/hpg_res_store.csv', encoding='latin1')

In [ ]:
hpginfo.tail()

In [ ]:
airinfo.tail()

In [ ]:
airinfo.tail()

In [ ]:
import time
area_list = {'air':[], 'hpg':[]} # initialize a dict to store restaurants with same area
area_temp = area.set_index('air_area', drop=False)  # a temporary area table, using it to find index
start = time.clock()
for i in range(0,len(area.air_area)):  # loop for areas of air restaurant 
    area_list[i] = {'air':[], 'hpg':[]}   # an area list, store restaurants with the same area
    air_area_name = area.air_area[i]    # get the area name
    for j in range(0,len(airinfo.air_store_id)):  
        air = airinfo.air_store_id[j]
        if (airinfo.loc[j, 'air_area_name'] == air_area_name):
            for p in range(0,len(hpginfo.hpg_store_id)):
                hpg = hpginfo.hpg_store_id[p]
                hpg_area = hpginfo.loc[p, 'hpg_area_name']
                if (area_temp.loc[air_area_name, hpg_area] == 1 or air_area_name == hpg_area):
                    area_list[i]['air'].append(air)
                    area_list[i]['hpg'].append(hpg)
                    end = time.clock() - start
                    print('Time costed:', end)
    print('No. %d area' %i)                             


In [ ]:
num = 0
air_sum = 0
hpg_sum = 0
for i in range(0,len(area.air_area)):
    if (len(area_list[i]['air']) != 0 ):
        area_list[i]['air'] = list(set(area_list[i]['air']))
        air_len = len(area_list[i]['air'])
        area_list[i]['hpg'] = list(set(area_list[i]['hpg']))
        hpg_len = len(area_list[i]['hpg'])
        air_sum = air_sum + air_len
        hpg_sum = hpg_sum + hpg_len
        num += 1
print('Total matching areas:', num)
print('Total air matching restaurants:', air_sum)
print('Total hpg matching restaurants:', hpg_sum)

In [ ]:
area_list

In [ ]:
# match latitude and longitude
start = time.clock()
for i in range(0, len(airinfo.air_store_id)):
    lalo_list[i] = {'hpg':[]}
    for j in range(0, len(hpginfo.hpg_store_id)):
        if (abs(airinfo.latitude[i] - hpginfo.latitude[j]) <= 0.25 and abs(airinfo.longitude[i] - hpginfo.longitude[j]) <= 0.05):
            lalo_list[i]['hpg'].append(hpginfo.hpg_store_id[j])
            print('Matching No.%d air restaurant and No.%d hpg' %(i, j))
end = (time.clock() - start) / 60
print('Time costed: %s minutes' %end)

In [ ]:
num = 0
for i in range(0,len(lalo_list) - 2):
    if (len(lalo_list[i]['hpg']) == 0):
        num += 1
        print(i)
print(num)

In [ ]:
# matching area and latitude/longitude
for i in range(0, len(airinfo.air_store_id)):
    for j in range(0, len(area_list) - 2):
        for p in range(0, len(area_list[j]['air'])):
            if (area_list[j]['air'][p] == airinfo.air_store_id[i] and len(lalo_list[i]['hpg']) != 0):
                lalo_list[i]['hpg'] = list(set(lalo_list[i]['hpg']).intersection(set(area_list[j]['hpg'])))
            if (area_list[j]['air'][p] == airinfo.air_store_id[i] and len(lalo_list[i]['hpg']) == 0):
                lalo_list[i]['hpg'] = list(set(lalo_list[i]['hpg']).union(set(area_list[j]['hpg'])))
    print('Calculating No.%d air restaurant' %i)

In [ ]:
airid = airinfo.air_store_id
area_match = lalo_list
genre_temp = genre.set_index('air_genre', drop=False)
genre_hpg = hpginfo.set_index('hpg_store_id', drop=False)
genre_match = {'hpg':[]}
for i in range(0, len(airid)):
    genre_match[i] = {'hpg':[]}
    airgenre = airinfo.loc[i, 'air_genre_name']
    for j in range(0, len(area_match[i]['hpg'])):
        hpggenre = genre_hpg.loc[area_match[i]['hpg'][j], 'hpg_genre_name']
        if (genre_temp.loc['genre_'+airgenre,'genre_'+hpggenre] == 1):
            genre_match[i]['hpg'].append(area_match[i]['hpg'][j])
    print('Matching No.%d air restaurant' %i)

In [ ]:
genre_match

In [ ]:
import numpy as np
import time
import fastdtw
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
start_time = time.clock()
air_temp = air_res_vis_store.set_index("air_store_id", drop=False)
hpg_temp = hpg_res_store.set_index('hpg_store_id', drop=False)
count = 0
airid = airinfo.air_store_id
dist = {'hpg':[], 'distance':[]}
for i in range(0,len(genre_match)-2):
    if (len(genre_match[i]['hpg']) != 0):
        # get air_visit visitors
        air_vis_list = air_temp.loc[airid[i],air_res_vis_store.columns[434]:air_res_vis_store.columns[911]]
        # initialize distance list
        dist[i] = {'hpg':[], 'distance':[]}
        # get hpg_reserve visitors
        for j in range(0,len(genre_match[i]['hpg'])):
            hpg_res_list = hpg_temp.loc[genre_match[i]['hpg'][j],hpg_res_store.columns[1]:hpg_res_store.columns[517]]
            distance, path = fastdtw(air_vis_list, hpg_res_list, dist=euclidean)
            dist[i]['distance'].append(distance)
            dist[i]['hpg'].append(genre_match[i]['hpg'][j])
    # get minimum distance
    min_dis = min(dist[i]['distance'])
    min_idx = dist[i]['distance'].index(min_dis)
    hpg_matching = dist[i]['hpg'][min_idx]
    end = (time.clock() - start_time) / 60
    # print results
    print('%s is matched with %s, DTW distance: %d' %(airid[i], hpg_matching, min_dis))
    print('Time costed: %s minutes' %end)
    print('\n')

In [ ]:
match_list = []
for m in range(0,len(re.air_store_id)):
    for n in range(0, len(airid)):
        if (airid[n] == re.air_store_id[m]):
            match_list.append(airid[n])

In [ ]:
len(match_list)